In [3]:
import pandas as pd
import math

In [9]:
class PickListMaker:
    
    def __init__(self, filenames, solution_moles, solution_volume, \
                 buffer_moles, origami=True, scaffold="M13mp18", ratio="10:1", solution_well="I10"):
        self.scaffold_molarity = {"M13mp18": 0.21e-12/1e-6}
        self.strands_df = self.generates_dataframes(filenames)
        self.solution_moles = solution_moles
        self.solution_volume = solution_volume
        self.origami = origami
        self.solution_well = solution_well
        self.buffer_moles = buffer_moles
        self.scaffold = scaffold
        self.ratio = float(ratio.split(":")[0]) if origami else 1
        self.picklist_df = self.generate_picklist(self.strands_df)

    def generates_dataframes(self, filenames):
        strands_dfs = []
        for filename in filenames:
            file_extn = filename.split(".")[-1]
            if file_extn == "xlsx": 
                strands_dfs.append(pd.read_excel(filename))
            elif file_extn == "csv":
                strands_dfs.append(pd.read_csv(filename))
        return strands_dfs

    def generate_picklist(self, all_staples):
        source_plate_name = []
        source_plate_type = []
        source_well_positions = []
        sample_comments = []
        destination_plate_name = []
        destination_wells = []
        transfer_volumes = []
        if self.origami:
            scaffold_transfer_volume = (self.solution_moles * self.solution_volume) / self.scaffold_molarity[self.scaffold]
        else:
            scaffold_transfer_volume = 0
        staple_concentrations = [staples_group["Measured Concentration µM "].mean() for staples_group in all_staples]
        staples_num = sum([len(staples_group) for staples_group in all_staples])
        staple_concentrations_avg = math.floor(sum(staple_concentrations) / len(staple_concentrations))*1e-6
        staple_transfer_volume = (self.ratio * 
                                  self.solution_moles * self.solution_volume) / staple_concentrations_avg
        staple_transfer_volume = int(round(staple_transfer_volume*1e+9, 0)) # in nanoliters
        dna_volume = scaffold_transfer_volume + staples_num * staple_transfer_volume*1e-9
        buffer_volume = self.solution_volume - dna_volume
        magnesium_volume = self.solution_volume * self.buffer_moles
        print("Average Staple Concetration is: {:.1f}µl with transfer volume {:.1f}nl.".format(round(staple_concentrations_avg*1e+6, 0), staple_transfer_volume))
        print("Total DNA volume is {:.2f}µl and buffer volume is {:.2f}µl.".format(dna_volume*1e+6, buffer_volume*1e+6))
        print("Magnesium volume is {:.2f}µl. This will be placed in the last available well.".format(magnesium_volume*1e+6))
        for idx, staple_pkg in enumerate(all_staples):
            for item, row in staple_pkg.iterrows():
                source_plate_name.append(f"Source[{idx+1}]")
                source_plate_type.append("384PP_AQ_BP")
                last_well_position = row["Well Position"]
                source_well_positions.append(last_well_position)
                sample_comments.append(row["Sequence"])
                destination_plate_name.append("Destination[1]")
                destination_wells.append(self.solution_well)
                transfer_volumes.append(staple_transfer_volume)
                
        source_plate_name.append(f"Source[{idx+1}]")
        source_plate_type.append("384PP_AQ_BP")
        magnesium_well_position = last_well_position
        
        if int(magnesium_well_position[1:]) > 24:
            letter_ord = (ord(s.upper())+1 - 65) % 26 + 65
            if letter_ord > 80:
                print("Ran out of well to place buffer, please pipet manually.")
            else:
                magnesium_well_position[0] = chr(letter_ord)
        else:
            magnesium_well_position = magnesium_well_position[0] + str(int(magnesium_well_position[1:]) + 1).zfill(2)
                                            
            
        
        if magnesium_well_position != last_well_position:
            source_well_positions.append(magnesium_well_position)
            sample_comments.append("Mg")
            destination_plate_name.append("Destination[1]")
            destination_wells.append("I10")
            transfer_volumes.append(magnesium_volume*1e+9)
         
        if self.origami:
            source_plate_name.append(f"Source[{idx+1}]")
            source_plate_type.append("384PP_AQ_BP")
            scaffold_well_position = magnesium_well_position
            if int(scaffold_well_position[1:]) > 24:
                letter_ord = (ord(scaffold_well_position[0].upper())+1 - 65) % 26 + 65
                if letter_ord > 80:
                    print("Ran out of well to place buffer, please pipet manually.")
                else:
                    scaffold_well_position[0] = chr(letter_ord)
            else:
                scaffold_well_position = scaffold_well_position[0] + str(int(scaffold_well_position[1:]) + 1).zfill(2)


            if scaffold_well_position != magnesium_well_position:
                source_well_positions.append(scaffold_well_position)
                sample_comments.append(self.scaffold)
                destination_plate_name.append("Destination[1]")
                destination_wells.append("I10")
                transfer_volumes.append(scaffold_transfer_volume*1e+9)
#         print(len(source_plate_name), len(source_plate_type), len(source_well_positions), len(sample_comments), len(destination_plate_name), len(destination_wells), len(transfer_volumes))
        echo_df = pd.DataFrame({"Source Plate Name": source_plate_name, "Source Plate Type": source_plate_type, \
                           "Source Well": source_well_positions, "Sample Comments": sample_comments, \
                           "Destination Plate Name": destination_plate_name, "Destination Well": destination_wells, \
                           "Transfer Volume": transfer_volumes})
        echo_df["Transfer Volume"] = echo_df["Transfer Volume"].astype('int')
        echo_df = echo_df.set_index("Source Plate Name")
        return echo_df

In [10]:
# nanobricks protocol
# MgCl2 = 20nM
# Mix Moles = 1383 * 5nM
# Mix Volume = 50uL
mix_moles = 1383 * 5e-9
mix_volume = 50e-6
buffer_moles = 20e-9
picklist = PickListMaker(["~/Downloads/brick_plate_spec.xlsx"], \
                        mix_moles, mix_volume, buffer_moles, origami=False)

Average Staple Concetration is: 200.0µl with transfer volume 1729.0nl.
Total DNA volume is 2391.21µl and buffer volume is -2341.21µl.
Magnesium volume is 0.00µl. This will be placed in the last available well.


In [11]:
picklist.picklist_df

,Source Plate Type,Source Well,Sample Comments,Destination Plate Name,Destination Well,Transfer Volume
Source Plate Name,,,,,,
Source[1],384PP_AQ_BP,A01,GAA CAC CGG ACT AAC GCC AGC CGC AGA AGA GAT TC...,Destination[1],I10,1729
Source[1],384PP_AQ_BP,A02,TTA TAC ATG CGG CCC GAC AAC CCA GCA ACA TTC GT...,Destination[1],I10,1729
Source[1],384PP_AQ_BP,A03,CCG TCG TGG TAT ATG GAA TCA GGG CGC CTT GTC TA...,Destination[1],I10,1729
Source[1],384PP_AQ_BP,A04,CTG ATG CTC GAA CAT AGC GGT CGA ATA TTC ATG GC...,Destination[1],I10,1729
Source[1],384PP_AQ_BP,A05,GGA AGC CGC TCC AGA CGC GGT CGT TCA AAA CTA TC...,Destination[1],I10,1729
...,...,...,...,...,...,...
Source[1],384PP_AQ_BP,P21,CAA GGA AGC GAT CTT TTT TTT TTA CTC TGC CCT TAA,Destination[1],I10,1729
Source[1],384PP_AQ_BP,P22,GGC CTC AGG TAC CTT TTT TTT TTA GGT AGG AGC TCG,Destination[1],I10,1729
Source[1],384PP_AQ_BP,P23,TCT TTC ACG GTA ATT TTT TTT TTA TGC ACC AGA TTT,Destination[1],I10,1729


In [32]:
df = pd.read_excel("~/Downloads/brick_plate_spec.xlsx")
df

,Plate Name,Payment Method,Plate Barcode,Sales Order #,Reference #,Well Position,Sequence Name,Sequence,Manufacturing ID,Measured Molecular Weight,Calculated Molecular Weight,OD260,nmoles,µg,Measured Concentration µM,Final Volume µL,Extinction Coefficient L/(mole·cm),Tm,Well Barcode
0,brick_sequences2 (1).xls,OligoCard,16853869,18812477,417199094,A01,STRAND-0398,GAA CAC CGG ACT AAC GCC AGC CGC AGA AGA GAT TC...,507145978,15920.3,15921,6.53,13.08,208,201.23,65,499400,71,NaN
1,brick_sequences2 (1).xls,OligoCard,16853869,18812477,417199095,A02,STRAND-0415,TTA TAC ATG CGG CCC GAC AAC CCA GCA ACA TTC GT...,507146170,15819.3,15820,6.36,12.84,203,200.63,64,495100,69,NaN
2,brick_sequences2 (1).xls,OligoCard,16853869,18812477,417199096,A03,STRAND-0431,CCG TCG TGG TAT ATG GAA TCA GGG CGC CTT GTC TA...,507145979,16066.7,16067,6.66,13.14,211,202.15,65,507100,68,NaN
3,brick_sequences2 (1).xls,OligoCard,16853869,18812477,417199097,A04,STRAND-0447,CTG ATG CTC GAA CAT AGC GGT CGA ATA TTC ATG GC...,507146171,16067.2,16068,6.50,12.99,209,202.97,64,500200,70,NaN
4,brick_sequences2 (1).xls,OligoCard,16853869,18812477,417199098,A05,STRAND-0464,GGA AGC CGC TCC AGA CGC GGT CGT TCA AAA CTA TC...,507145980,15949.6,15950,6.35,13.11,209,201.69,65,484600,73,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378,brick_sequences3 (1),OligoCard,16863945,18812477,417199858,P20,STRAND-1104,ACA ATT GTG TAG GTT TTT TTT TTG AGA CCG ACA GGA,507146743,11135.5,11135,4.56,12.89,144,201.41,64,353500,62,NaN
1379,brick_sequences3 (1),OligoCard,16863945,18812477,417199859,P21,STRAND-1120,CAA GGA AGC GAT CTT TTT TTT TTA CTC TGC CCT TAA,507146552,10966.6,10966,4.26,12.86,141,200.94,64,331200,61,NaN
1380,brick_sequences3 (1),OligoCard,16863945,18812477,417199860,P22,STRAND-1136,GGC CTC AGG TAC CTT TTT TTT TTA GGT AGG AGC TCG,507146744,11078.8,11079,4.32,12.98,144,202.81,64,333200,64,NaN
1381,brick_sequences3 (1),OligoCard,16863945,18812477,417199861,P23,STRAND-1152,TCT TTC ACG GTA ATT TTT TTT TTA TGC ACC AGA TTT,507146553,10962.5,10962,4.23,12.84,141,200.63,64,329500,58,NaN


In [16]:
picklist.picklist_df

,Source Plate Type,Source Well,Sample Comments,Destination Plate Name,Destination Well,Transfer Volume
Source Plate Name,,,,,,
Source[1],384PP_AQ_BP,A01,TCC AGT AAT ATA AGT CGT TAA TTC GAG ATT CGG TT...,Destination[1],I10,25
Source[1],384PP_AQ_BP,A02,CTA CTC ACT TTC CCA CTG CTA GGG CGA AGA GGG CA...,Destination[1],I10,25
Source[1],384PP_AQ_BP,A03,ATA CTG CGA ATG ACC ATT AAG GGT TTA CCG ATT CA...,Destination[1],I10,25
Source[1],384PP_AQ_BP,A04,AAC GCA AAG GAG CCA GCT TTC GTT CGA CTA T,Destination[1],I10,25
Source[1],384PP_AQ_BP,A05,ACA ACT CGC AAC CGA TGT CCG CGC AAG ACG TAA TC...,Destination[1],I10,25
...,...,...,...,...,...,...
Source[2],384PP_AQ_BP,P01,AAT TGT GTA AAA GAA GTG AT,Destination[1],I10,25
Source[2],384PP_AQ_BP,P02,GTA TGT GAA AAA GGA TAA GG,Destination[1],I10,25
Source[2],384PP_AQ_BP,P03,TAA GGT TAA AAA ATT GGA AT,Destination[1],I10,25
